## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


## Data import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [ ]:
explore = pd.read_csv("HomeCredit_columns_description.csv", encoding='latin1')
apl_train = pd.read_csv("application_train.csv")
apl_test = pd.read_csv("application_test.csv")
pos = pd.read_csv("POS_CASH_balance.csv")
bur = pd.read_csv("bureau.csv")
bur_bal = pd.read_csv("bureau_balance.csv")
credit = pd.read_csv("credit_card_balance.csv")
prev = pd.read_csv("previous_application.csv")
inst = pd.read_csv("installments_payments.csv")

## When merged, import train table

In [ ]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/balanced_train.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/destination_folder')  # Adjust the path and extraction folder

## Functions

In [ ]:
#  Performs statistical analysis on all dataframes

def perform_statistical_analysis(df):
    # Initialize a list to collect summary statistics
    summary_rows = []

    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column is numerical
        if df[column].dtype in [int, float]:
            # Compute descriptive statistics
            stats = df[column].describe()
            # Convert Series to a dictionary and add the column name
            stats_dict = stats.to_dict()
            stats_dict['Column'] = column
            # Calculate and add the number of missing values
            stats_dict['missing_count'] = df[column].isnull().sum()
            # Calculate and add the percentage of missing values
            stats_dict['% of missing'] = (stats_dict['missing_count'] / len(df)) * 100
            # Add the number of unique values
            stats_dict['unique_count'] = df[column].nunique()

            # Add the dictionary to the list of rows
            summary_rows.append(stats_dict)

    # Convert the list of dictionaries to a DataFrame
    summary_df = pd.DataFrame(summary_rows)

    # Reorder columns to have 'Column' first
    columns_order = ['Column', 'count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max', 'missing_count', '% of missing', 'unique_count']
    summary_df = summary_df[columns_order]

    # Transpose the DataFrame for better readability
    summary_df = summary_df.set_index('Column').transpose()

    # Display the summary DataFrame
    display(summary_df)

In [ ]:
def drop_cols(df, threshold=40):

    # Calculate the percentage of missing values for each column
    missing_count_pct = df.isnull().sum() / len(df) * 100

    # Drop columns where the percentage of missing values is greater than the threshold
    cols_to_drop = missing_count_pct[missing_count_pct > threshold].index
    df.drop(columns=cols_to_drop, inplace=True)

    return df

In [ ]:
def remove_duplicate_columns(df):
    # Transpose the DataFrame to compare columns as rows
    transposed_df = df.T

    # Drop duplicate rows in the transposed DataFrame, then transpose back
    deduplicated_df = transposed_df.drop_duplicates().T

    return deduplicated_df

In [ ]:
def cols_to_drop(df, threshold_1 = 25, threshold_2 = 5):

    cols_to_drop = []

    # Calculate the percentage of missing values for each column
    missing_count_pct = df.isnull().sum() / len(df) * 100

    # Loop through each column in the DataFrame
    for col in df.columns:
        # Check if the missing value percentage is above the threshold
        if missing_count_pct[col] > threshold_1:
            cols_to_drop.append(col)

        # Check if the column is of object type and has more unique values than the threshold
        if df[col].dtype == 'object' and df[col].nunique() > threshold_2:
            cols_to_drop.append(col)

    return cols_to_drop

In [ ]:
def correlation_with_value(df, target_column, value):
    # Check if the target column exists in the DataFrame
    if target_column not in df.columns:
        print(f"Column '{target_column}' does not exist in the DataFrame.")
        return

    # Check if the target column's data type is numeric
    if np.issubdtype(df[target_column].dtype, np.number):
        # Create a new DataFrame with only numeric columns
        numerical_df = df.select_dtypes(include=[np.number])

        # Compute the correlation matrix
        correlation_matrix = numerical_df.corr()

        # Ensure the target column exists in the correlation matrix
        if target_column in correlation_matrix.columns:
            # Compute correlation of each numeric column with the target column
            correlations = {}
            for col in numerical_df.columns:
                if col != target_column:
                    correlations[col] = df[[col, target_column]].corr().iloc[0, 1]

            # Convert correlations dictionary to a DataFrame for better readability
            correlations_df = pd.DataFrame(list(correlations.items()), columns=['Column', 'Correlation'])

            # Sort by correlation values
            correlations_df = correlations_df.sort_values(by='Correlation', ascending=False)
            return correlations_df

            # Print the correlation values
            print(f"Correlation of numeric columns with '{target_column}':")

            print(correlations_df)
        else:
            print(f"Column '{target_column}' is not present in the correlation matrix.")
    else:
        print(f"Column '{target_column}' is not numeric. Correlation analysis is not applicable.")

In [ ]:
def aggregate_data(data, groupby_column, label=None):
    ### SEPARATE FEATURES

    # Display info
    print("- Preparing the dataset...")

    # Find factors (categorical columns)
    data_factors = [f for f in data.columns if data[f].dtype == "object"]

    # Partition subsets
    num_data = data.drop(columns=data_factors)
    fac_data = data[[groupby_column] + data_factors]

    # Display info
    num_facs = fac_data.shape[1] - 1  # subtract 1 for the groupby column
    num_nums = num_data.shape[1]
    print(f"- Extracted {num_facs} factors and {num_nums} numerics...")

    ##### AGGREGATION

    # Aggregate numeric data
    if num_nums > 0:
        print("- Aggregating numeric features...")
        num_data = num_data.groupby(data[groupby_column]).agg(["mean", "std", "min", "max"])
        num_data.columns = ["_".join(col).strip() for col in num_data.columns.values]
        num_data = num_data.sort_index()

    # Aggregate categorical data
    if num_facs > 0:
        print("- Aggregating factor features...")
        fac_data = fac_data.groupby(groupby_column).agg([
            ("mode", lambda x: x.mode()[0] if not x.mode().empty else None),
            ("unique", lambda x: x.nunique())
        ])
        fac_data.columns = ["_".join(col).strip() for col in fac_data.columns.values]
        fac_data = fac_data.sort_index()

    ##### MERGER

    # Merge numeric and categorical data
    if num_nums > 0 and num_facs > 0:
        agg_data = pd.concat([num_data, fac_data], axis=1)
    elif num_nums > 0:
        agg_data = num_data
    elif num_facs > 0:
        agg_data = fac_data
    else:
        agg_data = pd.DataFrame()

    ##### LAST STEPS

    # Update labels if provided
    if label is not None:
        agg_data.columns = [f"{label}_{col}" for col in agg_data.columns]

    # Display info
    print("- Final dimensions:", agg_data.shape)

    # Return the aggregated dataset
    return agg_data

In [ ]:
def identify_log_transform_columns(df, skew_threshold=1.0):

    columns_to_transform = []

    for col in df.select_dtypes(include=[np.number]).columns:
        # Calculate the skewness
        skewness = df[col].skew()

        # Check if skewness is above the threshold
        if abs(skewness) > skew_threshold:
            columns_to_transform.append(col)

    return columns_to_transform

In [ ]:
def log_transform_columns(df, columns_to_transform):

    df_transformed = df.copy()
    for col in columns_to_transform:
        if col in df_transformed.columns:

            df_transformed[col] = df_transformed[col].apply(lambda x: np.log(x + 1) if x > 0 else 0)

    return df_transformed

In [ ]:

def high_correlation_columns(df, threshold):
    # Calculate the correlation matrix
    corr_matrix = df.corr().abs()  # Use absolute values to consider the magnitude

    # Create a mask to get pairs with correlation above the threshold
    # Exclude self-correlations by creating a mask for the upper triangle only
    mask = (corr_matrix > threshold) & (np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

    # Find the column pairs with high correlation
    high_corr_pairs = []
    for col in mask.columns:
        for row in mask.index:
            if mask.at[row, col]:
                high_corr_pairs.append((row, col, corr_matrix.at[row, col]))

    # Convert the list to a DataFrame for easier readability
    high_corr_df = pd.DataFrame(high_corr_pairs, columns=['Column1', 'Column2', 'Correlation'])

    return high_corr_df


In [ ]:
def balance_target_column(df, target_col="TARGET_mean", strategy="combine", random_state=42):

    # Split the features and target
    X = df.drop(columns=[target_col])
    y = df[target_col]

    # Choose the resampling strategy
    if strategy == "undersample":
        rus = RandomUnderSampler(random_state=random_state)
        X_resampled, y_resampled = rus.fit_resample(X, y)
    elif strategy == "oversample":
        smote = SMOTE(random_state=random_state)
        X_resampled, y_resampled = smote.fit_resample(X, y)
    else:  # combine undersampling and oversampling
        smote_enn = SMOTEENN(random_state=random_state)
        X_resampled, y_resampled = smote_enn.fit_resample(X, y)

    # Create a balanced DataFrame with the same feature columns and the resampled target column
    df_resampled = pd.DataFrame(X_resampled, columns=X.columns)
    df_resampled[target_col] = y_resampled

    return df_resampled

In [ ]:
def encode_categorical(df):
    df_encoded = df.copy()
    label_encoder = LabelEncoder()

    for col in df_encoded.columns:
        if df_encoded[col].dtype == 'object':
            df_encoded[col] = label_encoder.fit_transform(df_encoded[col])

    return df_encoded

In [ ]:
def resample_column_with_small_value(df, col, min_occurrence=0.01):

    if df[col].nunique() != 2:
        raise ValueError(f"The column '{col}' does not have exactly 2 unique values.")

    # Calculate value counts and their percentages
    value_counts = df[col].value_counts()
    total_count = len(df)
    value_percentages = value_counts / total_count * 100

    # Identify the less frequent value
    less_frequent_value = value_percentages.idxmin()
    less_frequent_percentage = value_percentages.min()

    if less_frequent_percentage < min_occurrence:
        print(f"Column '{col}' has a value '{less_frequent_value}' with a percentage of {less_frequent_percentage:.2f}%. Resampling this column.")

        # Define a new value to replace the less frequent one (e.g., the more frequent value)
        more_frequent_value = value_percentages.idxmax()

        # Replace less frequent values with the more frequent value
        df[col] = df[col].replace(less_frequent_value, more_frequent_value)

    return df

In [ ]:
def fill_missing_values_by_digit_strategy(df):
    filled_df = df.copy()

    for col in filled_df.columns:
        # Select only numeric columns to process
        if pd.api.types.is_numeric_dtype(filled_df[col]):
            # Drop missing values to analyze actual values
            non_null_values = filled_df[col].dropna()

            # Check if the majority of the values in the column are single digits
            single_digit_ratio = (non_null_values.abs() < 10).mean()  # Proportion of single-digit values

            # Choose strategy based on the digit count
            if single_digit_ratio > 0.5:
                # Use mode if most values are single digits
                mode_value = non_null_values.mode()[0]
                filled_df[col].fillna(mode_value, inplace=True)
                print(f"Filled missing values in '{col}' using mode: {mode_value}")
            else:
                # Use mean if most values have two or more digits
                mean_value = non_null_values.mean()
                filled_df[col].fillna(mean_value, inplace=True)
                print(f"Filled missing values in '{col}' using mean: {mean_value}")

    return filled_df

In [ ]:
def fill_missing_with_lr(df):
    # Create a copy of the DataFrame to avoid modifying the original one
    df_filled = df.copy()

    # Identify columns with missing values
    target_cols = df.columns[df.isnull().any()].tolist()

    for col in target_cols:
        # Get the rows with missing values in the current column
        missing_rows = df_filled[df_filled[col].isnull()]

        for idx in missing_rows.index:
            # Row data without the target columns
            row_with_nan = df_filled.loc[idx].drop(labels=target_cols)
            X_missing = row_with_nan.values.reshape(1, -1)

            # Prepare the training data (rows without missing values in the current column)
            train_data = df_filled.dropna(subset=[col])
            X_train = train_data.drop(columns=target_cols)
            y_train = train_data[col]

            # Ensure there's enough data to train the model
            if len(X_train) > 1:
                # Initialize and train the linear regression model
                model = LinearRegression()
                model.fit(X_train, y_train)

                # Predict the missing value and fill it in the DataFrame
                predicted_value = model.predict(X_missing)
                df_filled.loc[idx, col] = predicted_value
            else:
                # If there's not enough data, we cannot train the model
                # You might want to fill with a different strategy (e.g., mean, median)
                df_filled.loc[idx, col] = df_filled[col].mean()  # Or any other strategy

    return df_filled

In [ ]:
def drop_uncommon_cols(df1, df2):
    # Find common columns in both DataFrames
    common_cols = list(set(df1.columns).intersection(set(df2.columns)))

    # Identify dropped columns
    dropped_cols_df1 = [col for col in df1.columns if col not in common_cols]
    dropped_cols_df2 = [col for col in df2.columns if col not in common_cols]

    # Retain only the common columns in both DataFrames
    df1 = df1[common_cols]
    df2 = df2[common_cols]

    return df1, df2, dropped_cols_df1, dropped_cols_df2

# Function call
train, test, dropped_train, dropped_test = drop_uncommon_cols(train, test)

# Print dropped columns
print("Dropped columns from train DataFrame:", dropped_train)
print("Dropped columns from test DataFrame:", dropped_test)

In [ ]:
# preprocessing step of aggregated and merged dataframe

def clean_dataframe(df, threshold=40):
    # Drop columns with missing values above a threshold
    df = drop_cols(df, threshold)

    # Encode categorical columns
    df = encode_categorical(df)

    # Fill missing values using a strategy (digit-based)
    df = fill_missing_values_by_digit_strategy(df)

    # Remove duplicate columns
    df = remove_duplicate_columns(df)

    # Identify columns for logarithmic transformation (non-normal distributions)
    check = identify_log_transform_columns(df)

    if check:
        user_input = input("Your dataframe contains columns with non-normal distribution. Would you like to logarithmically transform these columns (yes/no)? ").strip().lower()

        # Ensure valid input and proceed accordingly
        if user_input == "yes":
            df = log_transform_columns(df, check)
        elif user_input == "no":
            print("No transformations applied.")
        else:
            print("Invalid input. No transformations applied.")

    return df





In [ ]:
# ensure test and train dataframes are equal

def equalize_dataframes(df1, df2, y="Target_column"):
    if list(df1.columns) != list(df2.columns):  # Compare column names
        user_input = input("Columns are not equal, would you like to equalize? (yes/no): ").strip().lower()

        if user_input == "yes":
            # Find common columns in both DataFrames
            common_cols = list(set(df1.columns).intersection(set(df2.columns)))

            # Identify dropped columns
            dropped_cols_df1 = [col for col in df1.columns if col not in common_cols]
            dropped_cols_df2 = [col for col in df2.columns if col not in common_cols]

            # Retain only the common columns in both DataFrames
            df1 = df1[common_cols]
            df2 = df2[common_cols]

            print(f"Removed columns from df1: {dropped_cols_df1}")
            print(f"Removed columns from df2: {dropped_cols_df2}")
        elif user_input == "no":
            print("No changes made.")
        else:
            print("Invalid input. No changes made.")

    return df1, df2, y
